# Solutions to Exercises of Week 5
*These exercises will not be graded. Solutions will be made available, but it is strongly advised that you try on your own first.*

*Make sure that any function you write has a docstring, and comments where appropriate.*

## Question 1
The task is to implement a function `calltree_explicit(S0, K, T, r, sigma, N)` that computes the price of a European option based on the binomial distribution.

**1.1.** Import the relevant libraries:

In [1]:
import numpy as np
from scipy.stats import binom
from scipy.optimize import brentq

**1.2.** Set the option parameters:

In [2]:
S0=11.; K=10.; T=3/12.; r=.02; sigma=.3; N=500;

**1.3.** Define the function `calltree_explicit(S0, K, T, r, sigma, N)`. 

*Hints*:
* The survivor function of the binomial distribution is available as `binom.sf` from `scipy.stats`.
* Numpy has a `ceil` function that returns the smallest integer greater than a real number.

In [3]:
def calltree_explicit(S0, K, T, r, sigma, N):
    """European call price based on an N-step binomial tree."""
    deltaT = T/float(N)
    u = np.exp(sigma * np.sqrt(deltaT))
    d = 1/u
    p = (np.exp(r*deltaT) - d)/(u-d)   
    pstar = np.exp(-r*deltaT)*p*u
    a = np.ceil(N/2+np.log(K/S0)/(2*np.log(u)))
    return S0*binom.sf(a-1, N, pstar)-binom.sf(a-1, N, p)*np.exp(-r*T)*K

**1.4.** Copy the function `calltree_numpy` from the slides for comparison:

In [4]:
def calltree_numpy(S0, K, T, r, sigma, N):
    """European call price based on an N-step binomial tree."""
    deltaT = T/float(N)
    u = np.exp(sigma * np.sqrt(deltaT))
    d = 1/u
    p = (np.exp(r*deltaT) - d)/(u-d)
    piu = np.exp(-r*deltaT)*p
    pid = np.exp(-r*deltaT)*(1-p)
    C = np.zeros((N+1, N+1))
    S = S0 * u**np.arange(N+1) * d**(2*np.arange(N+1)[:, np.newaxis])
    S = np.triu(S)  #Keep only the upper triangular part.
    C[:, N] = np.maximum(0, S[:, N]-K) #Note: np.maximum in place of max.
    for j in range(N-1, -1, -1):
        C[:j+1, j] = piu * C[:j+1, j+1] + pid * C[1:j+2, j+1]
    return  C[0, 0]

**1.5.** Verify that both implementations give the same result:

In [5]:
np.allclose(calltree_numpy(S0, K, T, r, sigma, N), calltree_explicit(S0, K, T, r, sigma, N))

True

**1.6.** Time both functions to see which is faster:

In [6]:
%timeit calltree_numpy(S0, K, T, r, sigma, N)

12.2 ms ± 777 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
%timeit calltree_explicit(S0, K, T, r, sigma, N)

388 µs ± 64 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Question 2

**2.1.** Write a function `puttree_american_numpy(S0, K, T, r, sigma, N)` that computes the price of an American put, by modifying `calltree_numpy(S0, K, T, r, sigma, N)`:

In [8]:
def puttree_american_numpy(S0, K, T, r, sigma, N):
    """American put price based on an N-step binomial tree."""
    deltaT = T/float(N)
    u = np.exp(sigma * np.sqrt(deltaT))
    d = 1/u
    p = (np.exp(r*deltaT) - d)/(u-d)
    piu = np.exp(-r*deltaT)*p
    pid = np.exp(-r*deltaT)*(1-p)
    
    P = np.zeros((N+1,N+1))
    
    S = S0 * u**np.arange(N+1) * d**(2*np.arange(N+1)[:, np.newaxis])
    S = np.triu(S)
    
    P[:, N]=np.maximum(0, K-S[:, N])
    
    for j in range(N-1, -1, -1):
        P[:j+1, j] = piu * P[:j+1, j+1] + pid * P[1:j+2, j+1]
        P[:j+1, j] = np.maximum(P[:j+1, j], K-S[:j+1, j] )
    
    return  P[0, 0]

In [9]:
puttree_american_numpy(S0, K, T, r, sigma, N)

0.2369178713067898

**2.2.** Make a function `impvol_puttree(S0, K, T, r, C_obs, N)` that computes the tree-implied volatility for an American put option.

*Hint*: The Theranchi bounds don't apply here. Use $L=0.1$ and $U=10$.

In [10]:
def impvol_puttree(S0, K, T, r, P_obs, N):
    """Tree-implied volatility for American puts."""
    L = 0.1
    U = 10
    return brentq(lambda s: puttree_american_numpy(S0, K, T, r, s, N)-P_obs, L, U)

In [11]:
P_obs=0.3
impvol_puttree(S0, K, T, r, P_obs, N)

0.3367514414348106